## D'NOVA YouTube Shorts Comments collection
- selenium을 통해 쇼츠 커멘트 수집
- 기본적으로 shorts url은 전부 가지고 있다고 가정

In [5]:
!pip install undetected-chromedriver

In [6]:
import time
import random
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd

In [7]:
def random_sleep():
    """ 1~2초 사이의 랜덤한 대기 """
    time.sleep(random.uniform(1, 2))

def scroll_to_bottom(driver):
    """ 페이지를 끝까지 스크롤 다운하는 함수 """
    last_height = driver.execute_script("return document.documentElement.scrollHeight")
    
    while True:
        # Scroll down to bottom
        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
        random_sleep()  # 랜덤한 시간 대기
        
        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.documentElement.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

def get_youtube_comments(url):
    # uc driver로 브라우저 실행
    driver = uc.Chrome()
    driver.get(url)
    random_sleep()  # 페이지가 로드될 시간을 줍니다.
    
    # 페이지 끝까지 스크롤 다운
    scroll_to_bottom(driver)
    
    # 모든 댓글 수집
    comments_elements = driver.find_elements(By.XPATH, '//*[@id="content-text"]/span')
    comments = [comment.text for comment in comments_elements]
    random_sleep()  # 데이터 수집 중간에도 랜덤 대기
    
    # 댓글 작성 날짜 수집
    date_elements = driver.find_elements(By.XPATH, '//*[@id="published-time-text"]/a')
    dates = [date.text for date in date_elements]
    random_sleep()  # 데이터 수집 중간에도 랜덤 대기
    
    # 결과를 리스트의 튜플 형태로 반환
    comments_data = list(zip(comments, dates))
    
    # 드라이버 종료
    driver.quit()
    
    return comments_data

# 예시 URL
url = "https://www.youtube.com/watch?v=xEuH_5Ik92U"
comments_list = get_youtube_comments(url)

# 리스트와 튜플 형태의 데이터 출력
print(comments_list)


                                               Comment   Date
0               BGM궁금해하시는 분들\n많은데 "your new home" 입니다!   2주 전
1    스피드 진짜 어떤면에선 미@친놈인데 진짜 이런거 보면 진짜 훈훈하고 스피드 진짜 멋...   2주 전
2                      진짜 편견없이 바라봐주는구나 이런거보면 진짜 존나 착한듯   2주 전
3    인격적으로 채우는 모습. 장애인분들 표정에 행복한 감정이 분명하게 느껴진다. 계속 ...  13일 전
4                                       첫번째 여자팬은 진짜귀여움   2주 전
..                                                 ...    ...
240                                              얘가몬대?   7일 전
241                                               꺼무하네  11일 전
242                                    이런게 뭐가좋다고 열광하는지   2주 전
243                                                 3빠   2주 전
244                 청각장애인척하고 스피드한테 돈뜯으려고 하는 개수작으로 보이는데   2주 전

[245 rows x 2 columns]
